In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, decomposition, ensemble
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold,StratifiedShuffleSplit,KFold,ShuffleSplit,train_test_split
from sklearn.feature_selection import SelectKBest, chi2
# import xgboost, numpy, textblob, string
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers
# from gensim import parsing 
from collections import Counter
from nltk import ngrams
import numpy as np
import pandas as pd
import multiprocessing as mp
from joblib import Parallel, delayed
from tqdm import tqdm
from datetime import date, timedelta
from IPython.core.display import display, HTML
import requests, json, os, time, datetime, random, glob, re, csv, urllib, nltk, collections, pickle, random
from bs4 import BeautifulSoup as bs
from bs4.element import Tag
from dateutil.parser import parse
from stop_words import get_stop_words
from string import digits
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.util import everygrams
from nltk.tokenize import sent_tokenize, RegexpTokenizer

def rmnl(t):
    return re.sub('\s+',' ',t).strip()

def map_filenolocs(x,filelocdict):
    cleanjl = []
    for k,v in filelocdict.items():
        prfxs = [i for i in v if i in x]
        if len(prfxs) >0:
            cleanjl.append(v[0])
        else:
            continue
    return '|'.join(list(set(cleanjl)))

def get_wordcount(x):
    return len(x.split())

mainpath = str(os.getcwd())+'/'

df = pd.read_csv(mainpath +'output/4_traindataplus.csv',sep='\t')
df['appcleantext1'] = df['cleantext'].apply(lambda x: ' '.join(x.split(' ')[:100]))
df['wincleantext1'] = df['cleantext'].apply(lambda x: ' '.join(x.split(' ')[-round(0.5*len(x.split(' '))):]).strip())
df['appcleantext2'] = df['topiccleantext'].apply(lambda x: ' '.join(x.split(' ')[:100]))
df['wincleantext2'] = df['topiccleantext'].apply(lambda x: ' '.join(x.split(' ')[-round(0.5*len(x.split(' '))):]).strip())
print(df.shape)

dftraintest = df[df['outcome']!= 'nottrainingsample']
dftraintest = dftraintest.copy()
df0 = df[df['outcome']== 'nottrainingsample']
dftrain = dftraintest.head(1100)
dftest = dftraintest[~dftraintest['cid'].isin(dftrain['cid'].tolist())]
print(dftraintest.shape)
print(dftrain.shape)
print(dftest.shape)
print(df0.shape)

dftraintest['traintest'] = np.where(dftraintest['cid'].isin(dftrain['cid'].tolist()),'train','test')
print(dftraintest['traintest'].value_counts())

(36461, 33)
(1636, 33)
(1100, 33)
(536, 33)
(34825, 33)
train    1100
test      536
Name: traintest, dtype: int64


<h2> APPLICANTS </h2>

In [2]:
paramgrid = {'C': [1000,100,10,1],
                  'gamma': [10,1, 0.1, 0.01,0.001], 
                  'kernel': ['rbf','poly','sigmoid','linear'],
                 "class_weight":["balanced",None]}

# cvectorizer = CountVectorizer(ngram_range=(1,3), max_features=100,min_df=5)
cvectorizera = CountVectorizer(ngram_range=(2,3))

ssp = ShuffleSplit(n_splits = 5, test_size=0.3, train_size=0.3,random_state=0) # instead of folding, just shuffling for each train/test sample (5 splits)
encoder = preprocessing.LabelEncoder()

xa = dftrain['appcleantext'].tolist()
ya = dftrain['applicant'].tolist()

ya = encoder.fit_transform(ya)
cvectorizera.fit(xa)
xa = cvectorizera.transform(xa)#.reshape(-1, 1)

selectora = SelectKBest(chi2, k=100)
selectora.fit(xa, ya)
xa = selectora.transform(xa)

In [3]:
grida = GridSearchCV(svm.SVC(probability=True), paramgrid, cv=ssp, refit = True, verbose = 3) # cv=10, refit = True,
grida.fit(xa, ya)

Fitting 5 folds for each of 160 candidates, totalling 800 fits
[CV 1/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.733 total time=   0.1s
[CV 2/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.685 total time=   0.1s
[CV 3/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.676 total time=   0.1s
[CV 4/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.700 total time=   0.1s
[CV 5/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.706 total time=   0.1s
[CV 1/5] END C=1000, class_weight=balanced, gamma=10, kernel=poly;, score=0.988 total time=   0.0s
[CV 2/5] END C=1000, class_weight=balanced, gamma=10, kernel=poly;, score=0.970 total time=   0.0s
[CV 3/5] END C=1000, class_weight=balanced, gamma=10, kernel=poly;, score=0.967 total time=   0.0s
[CV 4/5] END C=1000, class_weight=balanced, gamma=10, kernel=poly;, score=0.967 total time=   0.0s
[CV 5/5] END C=1000, class_weight=balanced, gamma=1

GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=0, test_size=0.3, train_size=0.3),
             estimator=SVC(probability=True),
             param_grid={'C': [1000, 100, 10, 1],
                         'class_weight': ['balanced', None],
                         'gamma': [10, 1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             verbose=3)

In [4]:
xat = dftest['appcleantext'].tolist()
yat = dftest['applicant'].tolist()

yat = encoder.fit_transform(yat)
xat = cvectorizera.transform(xat)#.reshape(-1, 1)

xat = selectora.transform(xat)

In [5]:
print(grida.best_params_)  
print(grida.best_estimator_) 

predictionsa = grida.predict(xat)

print(classification_report(yat, predictionsa)) 

{'C': 10, 'class_weight': None, 'gamma': 0.01, 'kernel': 'sigmoid'}
SVC(C=10, gamma=0.01, kernel='sigmoid', probability=True)
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       381
           1       0.96      0.97      0.97       155

    accuracy                           0.98       536
   macro avg       0.98      0.98      0.98       536
weighted avg       0.98      0.98      0.98       536



In [6]:
xao = dftraintest['appcleantext'].tolist()
yao = dftraintest['applicant'].tolist()

yao = encoder.fit_transform(yao)
xao = cvectorizera.transform(xao)#.reshape(-1, 1)

xao = selectora.transform(xao)

svm_model_applicant = svm.SVC(C= 10, class_weight= None, gamma= 0.01, kernel = 'sigmoid', probability=True) # Linear Kernel

svm_model_applicant.fit(xao, yao)

SVC(C=10, gamma=0.01, kernel='sigmoid', probability=True)

In [7]:
""" predict """

Xa = df['appcleantext'].tolist()
Xa = cvectorizera.transform(Xa)
applicant_vect = selectora.transform(Xa)

y_applicant = svm_model_applicant.predict(applicant_vect)
df['applicant'] = y_applicant 

print(df['applicant'].value_counts())

0    30601
1     5860
Name: applicant, dtype: int64


In [39]:
tsamp = random.sample(df[df['applicant']==1]['text'].tolist(),1)[0][:1000]
print(tsamp)

Order under Section 30 Residential Tenancies Act, 2006 File Number: TET-33869-13 M.B., T.B., C.H., S.R. and B.B. (the 'Tenants') applied for an order determining that W.C. (the 'Landlord') failed to meet the Landlord's maintenance obligations under the Residential Tenancies Act, 2006 (the 'Act') or failed to comply with health, safety, housing or maintenance standards. This application was heard in Lindsay on March 28, 2013. The Tenants (C.H.) and the Landlord’s legal representative M.P. attended the hearing. The Tenants were assisted by duty counsel S.C. At the hearing, the parties consented to the following order. It is ordered that: 1. The Landlord will pay to the Tenants the sum of $300.00. This amount will be paid on March 28, 2013. 2. This order settles all matters arising from this tenancy to date and neither party will file any other application, action, claim or commence any proceedings against the other party related to this tenancy in any court of competent jurisdiction. Apr

In [40]:
with open(mainpath +'models/applicant.pkl', 'wb') as f:
    pickle.dump((cvectorizera, selectora, grida, svm_model_applicant), f)

# with open(mainpath +'models/applicant.pkl', "rb") as f:
#      cvectorizera, selectora, grida, svm_model_applicant = pickle.load(f)

<h2> OUTCOMES </h2>

In [41]:
cvectorizerw = CountVectorizer(ngram_range=(2,3))

xw = dftrain['wincleantext2'].tolist()
yw = dftrain['winner'].tolist()

yw = encoder.fit_transform(yw)
cvectorizerw.fit(xw)
xw = cvectorizerw.transform(xw)#.reshape(-1, 1)

selectorw = SelectKBest(chi2, k=1000)
selectorw.fit(xw, yw)
xw = selectorw.transform(xw)

In [42]:
gridw = GridSearchCV(svm.SVC(probability=True), paramgrid, cv=ssp, refit = True, verbose = 3) # cv=10, refit = True,
gridw.fit(xw, yw)

Fitting 5 folds for each of 160 candidates, totalling 800 fits
[CV 1/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.594 total time=   0.2s
[CV 2/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.570 total time=   0.2s
[CV 3/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.606 total time=   0.2s
[CV 4/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.591 total time=   0.2s
[CV 5/5] END C=1000, class_weight=balanced, gamma=10, kernel=rbf;, score=0.627 total time=   0.2s
[CV 1/5] END C=1000, class_weight=balanced, gamma=10, kernel=poly;, score=0.815 total time=   0.1s
[CV 2/5] END C=1000, class_weight=balanced, gamma=10, kernel=poly;, score=0.752 total time=   0.1s
[CV 3/5] END C=1000, class_weight=balanced, gamma=10, kernel=poly;, score=0.761 total time=   0.1s
[CV 4/5] END C=1000, class_weight=balanced, gamma=10, kernel=poly;, score=0.833 total time=   0.1s
[CV 5/5] END C=1000, class_weight=balanced, gamma=1

GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=0, test_size=0.3, train_size=0.3),
             estimator=SVC(probability=True),
             param_grid={'C': [1000, 100, 10, 1],
                         'class_weight': ['balanced', None],
                         'gamma': [10, 1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid', 'linear']},
             verbose=3)

In [43]:
xwt = dftest['wincleantext2'].tolist()
ywt = dftest['winner'].tolist()

ywt = encoder.fit_transform(ywt)
xwt = cvectorizerw.transform(xwt)#.reshape(-1, 1)

xwt = selectorw.transform(xwt)

print(gridw.best_params_)  
print(gridw.best_estimator_) 

predictionsw = gridw.predict(xwt)

print(classification_report(ywt, predictionsw)) 

{'C': 10, 'class_weight': None, 'gamma': 0.001, 'kernel': 'rbf'}
SVC(C=10, gamma=0.001, probability=True)
              precision    recall  f1-score   support

           0       0.90      0.91      0.90       323
           1       0.86      0.85      0.85       213

    accuracy                           0.88       536
   macro avg       0.88      0.88      0.88       536
weighted avg       0.88      0.88      0.88       536



In [44]:
xwo = dftraintest['wincleantext2'].tolist()
ywo = dftraintest['winner'].tolist()

ywo = encoder.fit_transform(ywo)
xwo = cvectorizerw.transform(xwo)#.reshape(-1, 1)

xwo = selectorw.transform(xwo)

svm_model_outcome = svm.SVC(C= 10, class_weight= None, gamma= 0.001, kernel = 'rbf', probability=True) # Linear Kernel

svm_model_outcome.fit(xwo, ywo)

SVC(C=10, gamma=0.001, probability=True)

In [45]:
""" predict """

Xw = df['wincleantext2'].tolist()
Xw = cvectorizerw.transform(Xw)
winner_vect = selectorw.transform(Xw)

y_winner = svm_model_outcome.predict(winner_vect)
df['winner'] = y_winner 

print(df['winner'].value_counts())

0    27286
1     9175
Name: winner, dtype: int64


In [54]:
tsamp = random.sample(df[df['winner']==1]['text'].tolist(),1)[0][-1000:]
print(tsamp)

he months of January 2021 up to and including June 2021 in full, on or before the first day of each corresponding month. 3. If the Tenant fails to make any of the payments in accordance with paragraph 2, and by the dates required, then: (a) The Landlord may apply under section 78 of the Residential Tenancies Act, 2006 (the ‘Act’) for an order terminating the tenancy and evicting the Tenant, and for the payment of any new arrears of rent and NSF charges not already ordered under paragraph 1 of this order. The Landlord must make the application within 30 days of a breach of a condition set out in paragraph 2 of this order. (b) The balance owing under paragraph 1 of this order shall become payable on the day following the date of default. The monies owing shall bear interest at the post-judgment interest rate determined under subsection 207(7) of the Act. December 7, 2020 Date Issued Member, Landlord and Tenant Board South West-RO 150 Dufferin Avenue, Suite 400, 4th Floor London ON N6A5N6

In [55]:
with open(mainpath +'models/outcome.pkl', 'wb') as f:
    pickle.dump((cvectorizerw, selectorw, gridw, svm_model_outcome), f)
    
# with open(mainpath +'models/applicant.pkl', "rb") as f:
#      cvectorizerw, selectorw, gridw, svm_model_outcome = pickle.load(f)

In [56]:
print(df.columns)

Index(['cid', 'date', 'member', 'memberloc', 'filenos', 'loc', 'sections',
       'text', 'otext', 'error', 'caseurl', 'file', 'errtype', 'source',
       'jltext', 'cleantext', 'judgeloc', 'membertype', 'fileno',
       'fileno_clean', 'outcome', 'idxtext', 'appcleantext', 'wincleantext',
       'applicant', 'winner', 'filenoloc', 'newloc', 'topiccleantext',
       'appcleantext1', 'wincleantext1', 'appcleantext2', 'wincleantext2'],
      dtype='object')


<h2> TESTING OUTCOME MODEL ON FIRST 70% OF DOCUMENT </h2>

In [57]:
df['outcometest'] = df['cleantext'].apply(lambda x: ' '.join(x.split(' ')[:round(0.7*len(x.split(' ')))]).strip())

""" predict """

yt = df['winner'].tolist()
yt = encoder.fit_transform(yt)
Xt = df['outcometest'].tolist()
Xt = cvectorizerw.transform(Xt)
winnertest_vect = selectorw.transform(Xt)

y_winnertest = svm_model_outcome.predict(winnertest_vect)
df['winner_test'] = y_winnertest 

print(df['winner_test'].value_counts())

print(classification_report(yt, y_winnertest)) 

0    30619
1     5842
Name: winner_test, dtype: int64
              precision    recall  f1-score   support

           0       0.81      0.90      0.85     27286
           1       0.55      0.35      0.43      9175

    accuracy                           0.77     36461
   macro avg       0.68      0.63      0.64     36461
weighted avg       0.74      0.77      0.75     36461



In [58]:
dftraintest['outcometest'] = dftraintest['cleantext'].apply(lambda x: ' '.join(x.split(' ')[:round(0.7*len(x.split(' ')))]).strip())

""" predict """

yt = dftraintest['winner'].tolist()
yt = encoder.fit_transform(yt)
Xt = dftraintest['outcometest'].tolist()
Xt = cvectorizerw.transform(Xt)
winnertest_vect = selectorw.transform(Xt)

y_winnertest = svm_model_outcome.predict(winnertest_vect)
dftraintest['winner_test'] = y_winnertest 

print(dftraintest['winner_test'].value_counts())

print(classification_report(yt, y_winnertest)) 

0    1219
1     417
Name: winner_test, dtype: int64
              precision    recall  f1-score   support

           0       0.67      0.84      0.75       968
           1       0.63      0.40      0.49       668

    accuracy                           0.66      1636
   macro avg       0.65      0.62      0.62      1636
weighted avg       0.65      0.66      0.64      1636



In [59]:
selectorw_scores = pd.DataFrame(list(zip(cvectorizerw.get_feature_names(), selectorw.scores_, selectorw.pvalues_)), columns=['ftr', 'score', 'pval'])
selectorw_scores.sort_values(by=['score'],ascending=False, inplace= True)
selectora_scores = pd.DataFrame(list(zip(cvectorizera.get_feature_names(), selectora.scores_, selectora.pvalues_)), columns=['ftr', 'score', 'pval'])
selectora_scores.sort_values(by=['score'],ascending=False, inplace= True)

/home/bunds/bundslab/ltb_env/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [60]:
selectorw_scores.head(10)

,ftr,score,pval
75236,enforcement eviction enforce,406.178355,2.488913e-90
79986,eviction enforce,404.447768,5.925494e-90
75235,enforcement eviction,404.080381,7.123577e-90
123989,make motion,309.861018,2.341536e-69
148044,pay tenant,274.582959,1.137899e-61
75225,enforcement direct possession,253.587003,4.290051e-57
66664,direct possession unit,252.889703,6.087971e-57
75224,enforcement direct,251.247476,1.388293e-56
66661,direct possession,249.397824,3.513339e-56
164787,receipt enforcement direct,248.461688,5.620950e-56


In [61]:
selectora_scores.head(10)

,ftr,score,pval
34934,order determin,825.579452,1.481617e-181
3696,appli order determin,804.876173,4.697830e-177
51177,tenant appli,653.107466,4.713039e-144
51181,tenant appli order,640.779752,2.261285e-141
35002,order determin landlord,525.447730,2.762662e-116
13871,determin landlord,453.252502,1.413471e-100
40458,reason enjoy rental,383.335078,2.338141e-85
16247,enjoy rental,382.024499,4.510234e-85
16249,enjoy rental unit,379.895548,1.311300e-84
10685,complex tenant member,328.974609,1.607385e-73


In [62]:
selectora_scores.to_csv(mainpath+"models/5_applicantmodelfeatures.csv",sep='\t',index=False,quoting=csv.QUOTE_ALL)
selectorw_scores.to_csv(mainpath+"models/5_outcomemodelfeatures.csv",sep='\t',index=False,quoting=csv.QUOTE_ALL)

In [63]:
df.to_csv(mainpath+"output/5_model_results.csv",sep='\t',index=False,quoting=csv.QUOTE_ALL)
dftraintest.to_csv(mainpath+"models/5_dftraintestmodel_results.csv",sep='\t',index=False,quoting=csv.QUOTE_ALL)